In [40]:
#drawing block

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import gc
from sklearn import preprocessing
%matplotlib inline

def upload_file(fname):
    data = pd.read_csv("Our Dataset/" + fname,  index_col=False)
    return data

def make_columns_drawing(electrodes):
    newCols = []
    for col in electrodes:
        for i in range(192):
            newCols.append(col + '.' + str(i))
    newCols.append('y')
    return newCols

def convert_to_row(start, label, electrodes, data):
    row = []
    for col in electrodes:
        row += data[col][start:start+192].tolist()
    row += [label]
    return row

def positive_events(newCols, y, data, electrodes):
    posDfs = pd.DataFrame(columns = newCols)
    indices = []
    j = 0
    while j < (len(y)):
        #assume positive events are: pedestrian, dog, collision, and leadning car brake
        if (y[j] != 0):
            if (j+192 < len(y) and check_if_driver_brake(j, data)):
                posDfs.loc[len(posDfs)] = convert_to_row(j, 1, electrodes, data)
                indices.append(j)
                j += 384 #192*2
            else:
                j+=1
        else:
            j += 1
    return posDfs, indices  

def check_if_driver_brake(start_stim, data):
    for i in range(start_stim,start_stim+192):
        if(data["Brake Pedal "][i] == 1):
            return True
    return False

def negative_events(indices, newCols, y, data, electrodes):
    negDfs = pd.DataFrame(columns = newCols)
    j = 0
    while j < (len(y)):
        if(len(indices) > 0 and j <= indices[0] and j+192 > indices[0]):
            j = indices[0] + 384
            indices.pop(0)
        elif(j+192 < len(y)):
            negDfs.loc[len(negDfs)] = convert_to_row(j, 0, electrodes, data)
            j += 192
        else:
            break
    return negDfs



In [41]:
#run this block if you want to draw without preprocessing
def upload_and_draw(folder):
    fnames = os.listdir("Our Dataset/"+folder)
    data = upload_file(folder + '/'+fnames[0])
    data = data.drop("COUNTER", axis = 1)
    for i in range(1,len(fnames)):
        d = upload_file(folder + "/" + fnames[i])
        d = d.drop("COUNTER", axis = 1)
        data = data.append(d, ignore_index=True)
        print(fnames[i])
    #print(data.columns)
    electrodes = data.columns.drop(["Brake Pedal ", "y",])
    newCols = make_columns_drawing(electrodes)
    y = data["y"]
    posDfs, indices = positive_events(newCols, y, data, electrodes)
    posDfs.to_csv("Our Dataset/csv/"+folder+"pos.csv", index = False)
    negDfs = negative_events(indices, newCols, y, data, electrodes)
    negDfs.to_csv("Our Dataset/csv/"+folder+"neg.csv", index = False)
    meanPos, meanNeg = draw_signals(folder, posDfs, negDfs)

def draw_signals(subject, posDfs, negDfs):
    meanPos = np.mean(posDfs.drop("y", axis = 1))
    meanNeg = np.mean(negDfs.drop("y", axis = 1))
    cols = list(posDfs)
    for i in range(14):
        name = cols[i*192].split('.')[0]
        plt.ylim(np.amin(meanPos[i*192:i*192+192])-10, np.max(meanPos[i*192:i*192+192])+10)
        plt.title(subject + ": " + name, fontsize=16, fontweight='bold')
        plt.plot(meanPos[i*192:i*192+192].values, label="Braking Event")
        plt.plot(meanNeg[i*192:i*192+192].values, label="Normal Driving")
        plt.legend()
        plt.show()
    return meanPos, meanNeg


In [42]:
#run this block if you want to draw with preprocessing
def upload_and_draw(folder):
    fnames = os.listdir("Our Dataset/"+folder)
    data = upload_file(folder + '/'+fnames[0])
    data = data.drop("COUNTER", axis = 1)
    for i in range(1,len(fnames)):
        d = upload_file(folder + "/" + fnames[i])
        d = d.drop("COUNTER", axis = 1)
        data = data.append(d, ignore_index=True)
        print(fnames[i])
    electrodes = data.columns.drop(["Brake Pedal ", "y"])
    newCols = make_columns_drawing(electrodes)
    y = data["y"]
    posDfs, indices = positive_events(newCols, y, data, electrodes)
    posDfs.to_csv("Our Dataset/csv/"+folder+"pos.csv", index = False)
    negDfs = negative_events(indices, newCols, y, data, electrodes)
    negDfs.to_csv("Our Dataset/csv/"+folder+"neg.csv", index = False)
    meanPos, meanNeg = draw_signals(folder, posDfs, negDfs)
    return newCols

def draw_signals(subject, posDfs, negDfs):
    meanPos = preprocessing.scale(np.mean(posDfs.drop("y", axis = 1)))
    meanNeg = preprocessing.scale(np.mean(negDfs.drop("y", axis = 1)))
    cols = list(posDfs)
    for i in range(14):
        name = cols[i*192].split('.')[0]
        plt.ylim(np.amin(meanPos[i*192:i*192+192])-10, np.max(meanPos[i*192:i*192+192])+10)
        plt.title(subject + ": " + name, fontsize=16, fontweight='bold')
        plt.plot(meanPos[i*192:i*192+192], label="Braking Event")
        plt.plot(meanNeg[i*192:i*192+192], label="Normal Driving")
        plt.legend()
        plt.show()
    return meanPos, meanNeg



In [ ]:
upload_and_draw("folder name")

In [71]:
#accuracy computing 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn import preprocessing
from sklearn import metrics

def upload_file(fname):
    pos = pd.read_csv("Our Dataset/csv/" + fname + "pos" + ".csv", index_col=False)
    neg = pd.read_csv("Our Dataset/csv/" + fname + "neg" + ".csv", index_col=False)
    return pos, neg

def select_electrodes(electrodes, pos, neg):
    all_electrodes = pos.columns
    for i in range(0, 192*14, 192):
        electrode = all_electrodes[i].split('.')[0]
        if  electrode not in electrodes:
            pos = pos.drop([(electrode+'.'+str(j)) for j in range(192)], axis = 1)
            neg = neg.drop([(electrode+'.'+str(j)) for j in range(192)], axis = 1)
    return pos, neg

#put here your selected electrodes
electrodes = [' FC5']

all_electrodes = ['F3', ' FC5', ' AF3', ' F7', ' T7', ' P7', ' O1', ' O2', ' P8', ' T8',
       ' F8', ' AF4', ' FC6', ' F4']

cols = make_columns_drawing(electrodes)
newCols = make_columns_drawing(electrodes)

def make_columns(numOfCols):
    newCols = []
    for i in range(len(electrodes)):
        name = cols[i*192].split('.')[0]
        newCols.append(cols[i*192:i*192+numOfCols])
    return np.array(newCols).flatten()

def time_intervals_features(pos, neg, interval, numOfCols):
    #cols = list(pos)
    newCols = make_columns(numOfCols)
    mean_features_pos = pd.DataFrame(columns = newCols)
    mean_features_neg = pd.DataFrame(columns = newCols)
    cols = list(pos)
    step = int((len(cols)-1)/len(newCols))
    for i in range(0, len(electrodes)):
        for j in range(0, numOfCols):
            mean_features_pos[newCols[i*numOfCols + j]] = np.mean(pos[cols[i*interval+step*j:i*interval+numOfCols*j + step]], axis = 1).values
            mean_features_neg[newCols[i*numOfCols + j]] = np.mean(neg[cols[i*interval+step*j:i*interval+numOfCols*j + step]], axis = 1).values
            #print(mean_features_neg[newCols[i*numOfCols + j]])
    return mean_features_pos, mean_features_neg

def get_drop_cols(i1):
    dropCols = []
    nums = np.arange(i1, 192)
    for i in range(len(electrodes)):
        name = newCols[i*192].split('.')[0]
        dropCols.append([name + '.' + str(num) for num in nums])
    return np.array(dropCols).flatten()

def classify_diff_intervals_emotive():
    LDAClassifier = LinearDiscriminantAnalysis()
    fi = 0
    for f in fnames:
        print(f)
        for i in np.arange(12, 180, 12):
            pos, neg = upload_file(f)
            pos, neg = select_electrodes(electrodes, pos, neg)
            dropCols = get_drop_cols(i)
            #print(dropCols)
            pos = pos.drop(dropCols, axis = 1)
            neg = neg.drop(dropCols, axis = 1)
            mean_features_pos, mean_features_neg = time_intervals_features(pos, neg, i, int(i*6/12))
            mean_features_pos['y'] = 1
            mean_features_neg['y'] = 0
            all_data = mean_features_pos.append(mean_features_neg)
            y = all_data['y']
            X = all_data.drop(['y'], axis=1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1000)
            X_train = np.nan_to_num(X_train)
            X_test = np.nan_to_num(X_test)
            #X_train = preprocessing.scale(X_train)
            #X_test = preprocessing.scale(X_test)
            LDAClassifier.fit(X_train, y_train)
            y_predict = LDAClassifier.predict(X_test)
            #probs = LDAClassifier.predict_proba(X_test)
            #print(y_predict.shape)
            #print(probs.shape)
            #for i in range(len(probs)):
            #    print('{0:.10f}'.format(probs[i][0]))
            #    print('{0:.10f}'.format(probs[i][1]))
            print(str(i*7.8) + "ms")
            print(classification_report(y_test, y_predict))
            fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict)
            acc_auc = metrics.auc(fpr, tpr)
            print("AUC: " + str(acc_auc))
            print("-----------------------")
        fi += 1
    

In [6]:
fnames = ["folder name"]

In [ ]:
classify_diff_intervals_emotive()